# 9. Backtesting

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta

def create_sample_data(days=7, interval_hours=1):
   
    np.random.seed(42)
    n_points = int(days * 24 / interval_hours)
    dates = pd.date_range('2023-01-01', periods=n_points, freq=f'{interval_hours}h')
    close_prices = 0.05 + np.cumsum(np.random.randn(n_points) * 0.001)
    
    df = pd.DataFrame({
        'timestamp': dates,
        'open': close_prices - np.abs(np.random.randn(n_points) * 0.0005),
        'high': close_prices + np.abs(np.random.randn(n_points) * 0.0005),
        'low': close_prices - np.abs(np.random.randn(n_points) * 0.0005),
        'close': close_prices,
        'volume': np.random.rand(n_points) * 1000000
    })
    return df

def ema_touch_strategy_with_plot(coin, ema_number, budget, trading_fee=0.001, interval="1h", days=7, chart_height=800, trend="positive"):
    
   
    interval_hours = 1 if interval == "1h" else 4 if interval == "4h" else 24
    
 
    df = create_sample_data(days=days, interval_hours=interval_hours)

    ema_col = f"EMA_{ema_number}"
    df[ema_col] = df['close'].ewm(span=ema_number, adjust=False).mean()

    variance = (df['high'].max() - df['low'].min())
    take_profit_factor = variance / 8
    stop_loss_factor = take_profit_factor / 2

    trades = []
    current_budget = budget
    win_count = 0
    total_trades = 0
    active_trade = False
    entry_price = 0
    entry_time = None

    buy_markers = []
    sell_markers = []
    long_positions = []

    for i in range(1, len(df)):
        previous_price = df['close'].iloc[i - 1]
        current_price = df['close'].iloc[i]
        previous_ema = df[ema_col].iloc[i - 1]
        current_ema = df[ema_col].iloc[i]
        low_price = df['low'].iloc[i]
        high_price = df['high'].iloc[i]

        if trend == "positive":
            if not active_trade and previous_price > previous_ema and low_price <= current_ema:
                take_profit = current_price + take_profit_factor
                stop_loss = current_price - stop_loss_factor
                trade_amount = current_budget / current_price
                quantity_bought = trade_amount
                entry_price = current_price
                entry_time = df['timestamp'].iloc[i]
                active_trade = True
                total_trades += 1

                buy_markers.append((entry_time, entry_price))

            if active_trade:
                if high_price >= take_profit:
                    profit = quantity_bought * (take_profit - entry_price)
                    current_budget += profit - trading_fee * entry_price * quantity_bought
                    win_count += 1
                    trades.append({
                        "Trade Type": "Long",
                        "Buy Price": entry_price,
                        "Sell Price": take_profit,
                        "Quantity Bought": quantity_bought,
                        "Profit": profit,
                        "Loss": 0,
                        "Stop Loss": stop_loss,
                        "Entry Time": entry_time,
                        "Exit Time": df['timestamp'].iloc[i]
                    })
                    sell_markers.append((df['timestamp'].iloc[i], take_profit))
                    long_positions.append({
                        "entry_time": entry_time,
                        "exit_time": df['timestamp'].iloc[i],
                        "entry_price": entry_price,
                        "take_profit": take_profit,
                        "stop_loss": stop_loss,
                        "outcome": "profit"
                    })
                    active_trade = False

                elif low_price <= stop_loss:
                    loss = quantity_bought * (entry_price - stop_loss)
                    current_budget -= loss + trading_fee * entry_price * quantity_bought
                    trades.append({
                        "Trade Type": "Long",
                        "Buy Price": entry_price,
                        "Sell Price": stop_loss,
                        "Quantity Bought": quantity_bought,
                        "Profit": 0,
                        "Loss": loss,
                        "Stop Loss": stop_loss,
                        "Entry Time": entry_time,
                        "Exit Time": df['timestamp'].iloc[i]
                    })
                    sell_markers.append((df['timestamp'].iloc[i], stop_loss))
                    long_positions.append({
                        "entry_time": entry_time,
                        "exit_time": df['timestamp'].iloc[i],
                        "entry_price": entry_price,
                        "take_profit": take_profit,
                        "stop_loss": stop_loss,
                        "outcome": "loss"
                    })
                    active_trade = False

        elif trend == "negative":
            if not active_trade and previous_price < previous_ema and high_price >= current_ema:
                take_profit = current_price - take_profit_factor
                stop_loss = current_price + stop_loss_factor
                trade_amount = current_budget / current_price
                quantity_sold = trade_amount
                entry_price = current_price
                entry_time = df['timestamp'].iloc[i]
                active_trade = True
                total_trades += 1

                sell_markers.append((entry_time, entry_price))

            if active_trade:
                if low_price <= take_profit:
                    profit = quantity_sold * (entry_price - take_profit)
                    current_budget += profit - trading_fee * entry_price * quantity_sold
                    win_count += 1
                    trades.append({
                        "Trade Type": "Short",
                        "Sell Price": entry_price,
                        "Buy Back Price": take_profit,
                        "Quantity Sold": quantity_sold,
                        "Profit": profit,
                        "Loss": 0,
                        "Stop Loss": stop_loss,
                        "Entry Time": entry_time,
                        "Exit Time": df['timestamp'].iloc[i]
                    })
                    buy_markers.append((df['timestamp'].iloc[i], take_profit))
                    long_positions.append({
                        "entry_time": entry_time,
                        "exit_time": df['timestamp'].iloc[i],
                        "entry_price": entry_price,
                        "take_profit": take_profit,
                        "stop_loss": stop_loss,
                        "outcome": "profit"
                    })
                    active_trade = False

                elif high_price >= stop_loss:
                    loss = quantity_sold * (stop_loss - entry_price)
                    current_budget -= loss + trading_fee * entry_price * quantity_sold
                    trades.append({
                        "Trade Type": "Short",
                        "Sell Price": entry_price,
                        "Buy Back Price": stop_loss,
                        "Quantity Sold": quantity_sold,
                        "Profit": 0,
                        "Loss": loss,
                        "Stop Loss": stop_loss,
                        "Entry Time": entry_time,
                        "Exit Time": df['timestamp'].iloc[i]
                    })
                    buy_markers.append((df['timestamp'].iloc[i], stop_loss))
                    long_positions.append({
                        "entry_time": entry_time,
                        "exit_time": df['timestamp'].iloc[i],
                        "entry_price": entry_price,
                        "take_profit": take_profit,
                        "stop_loss": stop_loss,
                        "outcome": "loss"
                    })
                    active_trade = False

    win_rate = (win_count / total_trades) * 100 if total_trades > 0 else 0
    profit_loss = current_budget - budget

    trades_df = pd.DataFrame(trades) if trades else pd.DataFrame()

    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df['timestamp'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name="Candlestick",
        increasing=dict(line=dict(width=3)),
        decreasing=dict(line=dict(width=3))
    ))

    fig.add_trace(go.Scatter(
        x=df['timestamp'],
        y=df[ema_col],
        mode='lines',
        name=f"EMA {ema_number}",
        line=dict(color='blue', width=2)
    ))

    for pos in long_positions:
        color = "rgba(0,255,0,0.2)" if pos["outcome"] == "profit" else "rgba(255,0,0,0.2)"
        fig.add_shape(
            type="rect",
            x0=pos["entry_time"],
            x1=pos["exit_time"],
            y0=pos["stop_loss"],
            y1=pos["take_profit"],
            fillcolor=color,
            line=dict(width=0)
        )

    for timestamp, price in buy_markers:
        fig.add_trace(go.Scatter(
            x=[timestamp],
            y=[price],
            mode='markers',
            marker=dict(color='green', size=10),
            name="Buy Signal"
        ))

    for timestamp, price in sell_markers:
        fig.add_trace(go.Scatter(
            x=[timestamp],
            y=[price],
            mode='markers',
            marker=dict(color='red', size=10),
            name="Sell Signal"
        ))

    fig.update_layout(
        title=f"{coin} with EMA {ema_number} and Trades",
        xaxis_title="Timestamp",
        yaxis_title="Price",
        height=chart_height,
        template="plotly_dark",
        paper_bgcolor="black",
        plot_bgcolor="black"
    )

    return {
        "Number of Trades": total_trades,
        "Profit & Loss": profit_loss,
        "Win %": win_rate,
        "Ending Total Budget": current_budget,
        "Trades DataFrame": trades_df,
        "Candlestick Plot": fig
    }

In [11]:
result = ema_touch_strategy_with_plot(
    coin="GALAUSDT",
    ema_number=15,
    budget=3000,
    trading_fee=0.001,
    interval="1h",
    days=7,
    chart_height=700,
    trend="negative"
)

print(f"Number of Trades: {result['Number of Trades']}")
print(f"Profit & Loss: ${result['Profit & Loss']:.2f}")
print(f"Win %: {result['Win %']:.2f}%")
print(f"Ending Total Budget: ${result['Ending Total Budget']:.2f}")

print(result['Trades DataFrame'].head())

result['Candlestick Plot'].show()

Number of Trades: 11
Profit & Loss: $629.04
Win %: 54.55%
Ending Total Budget: $3629.04
  Trade Type  Sell Price  Buy Back Price  Quantity Sold      Profit  \
0      Short    0.051006        0.052203   58816.450229    0.000000   
1      Short    0.040562        0.038168   72150.085878  172.777279   
2      Short    0.039922        0.041119   77561.707994    0.000000   
3      Short    0.041708        0.039313   71939.521691  172.273042   
4      Short    0.039850        0.037455   79541.784417  190.478125   

        Loss  Stop Loss          Entry Time           Exit Time  
0  70.423660   0.052203 2023-01-01 02:00:00 2023-01-01 03:00:00  
1   0.000000   0.041760 2023-01-03 07:00:00 2023-01-03 14:00:00  
2  92.868225   0.041119 2023-01-03 17:00:00 2023-01-03 20:00:00  
3   0.000000   0.042905 2023-01-04 10:00:00 2023-01-04 23:00:00  
4   0.000000   0.041047 2023-01-05 02:00:00 2023-01-05 05:00:00  


In [12]:
result['Trades DataFrame']

,Trade Type,Sell Price,Buy Back Price,Quantity Sold,Profit,Loss,Stop Loss,Entry Time,Exit Time
0,Short,0.051006,0.052203,58816.450229,0.000000,70.423660,0.052203,2023-01-01 02:00:00,2023-01-01 03:00:00
1,Short,0.040562,0.038168,72150.085878,172.777279,0.000000,0.041760,2023-01-03 07:00:00,2023-01-03 14:00:00
2,Short,0.039922,0.041119,77561.707994,0.000000,92.868225,0.041119,2023-01-03 17:00:00,2023-01-03 20:00:00
3,Short,0.041708,0.039313,71939.521691,172.273042,0.000000,0.042905,2023-01-04 10:00:00,2023-01-04 23:00:00
4,Short,0.039850,0.037455,79541.784417,190.478125,0.000000,0.041047,2023-01-05 02:00:00,2023-01-05 05:00:00
5,Short,0.038763,0.036369,86603.671510,207.389174,0.000000,0.039961,2023-01-05 10:00:00,2023-01-05 14:00:00
6,Short,0.039699,0.040897,89701.530277,0.000000,107.403797,0.040897,2023-01-05 17:00:00,2023-01-06 00:00:00
7,Short,0.039759,0.037364,86776.343668,207.802670,0.000000,0.040956,2023-01-06 15:00:00,2023-01-06 22:00:00
8,Short,0.039389,0.036995,92777.712917,222.174105,0.000000,0.040587,2023-01-07 01:00:00,2023-01-07 03:00:00
9,Short,0.038248,0.039445,101259.983094,0.000000,121.243267,0.039445,2023-01-07 07:00:00,2023-01-07 13:00:00
